In [61]:
import tensorflow.compat.v1 as tf
# Using V1 API for better use with Elmo
tf.disable_v2_behavior()

import tensorflow_hub as hub
import numpy as np

# Using the Elmo framework from Google as our pre-trained model
# https://www.kaggle.com/models/google/elmo/frameworks/TensorFlow1/variations/elmo
embedding = hub.KerasLayer("https://www.kaggle.com/models/google/elmo/frameworks/TensorFlow1/variations/elmo/versions/3")

def find_category(categories_list, idea_str):
    """
    find_category outputs the category of a given idea string.

    :param categories_list: A list of possible categories (ex: ["carsharing","hardware","software"])
    :param idea_str: The string of the idea (ex: "Create an operating system easier to use than Windows.")
    :return: A dictionary with the matching category name in the "result" key and a "categories_scores" key with the raw scores.
    """

    # Prepare variables that will be locally using in the function
    categories_list_local = list(categories_list)
    categories_list_local.append(idea_str)
    num_categories = len(categories_list)

    similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
    similarity_message_encodings = embedding(similarity_input_placeholder)
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
        message_embeddings_ = session.run(similarity_message_encodings, feed_dict={similarity_input_placeholder: categories_list_local})

    categories_dict = {}

    # Loop over each category to check the relevance
    for i, category in enumerate(categories_list):
        score = np.inner(message_embeddings_[num_categories], message_embeddings_[i])
        categories_dict[category] = score

    # Find the highest value
    best_category = max(categories_dict, key=categories_dict.get)

    return ({"result" : best_category, "category_scores" : categories_dict})


categories_list = ["carsharing","medical","education","agriculture","software"]

idea_str = "Create an operating system easier to use than Windows."
results = find_category(categories_list, idea_str)
print ("Idea:", idea_str)
print ("Found category:", results["result"])
print ("Raw scores:", results["category_scores"])
print ()

idea_str = "A ride-sharing service for people with dogs."
results = find_category(categories_list, idea_str)
print ("Idea:", idea_str)
print ("Found category:", results["result"])
print ("Raw scores:", results["category_scores"])
print ()

idea_str = "A service to match tutors to students."
results = find_category(categories_list, idea_str)
print ("Idea:", idea_str)
print ("Found category:", results["result"])
print ("Raw scores:", results["category_scores"])
print ()

idea_str = "A medical device for better pulse measuring."
results = find_category(categories_list, idea_str)
print ("Idea:", idea_str)
print ("Found category:", results["result"])
print ("Raw scores:", results["category_scores"])
print ()

idea_str = "A low cost pesticide for paddy plantations."
results = find_category(categories_list, idea_str)
print ("Idea:", idea_str)
print ("Found category:", results["result"])
print ("Raw scores:", results["category_scores"])
print ()

Idea: Create an operating system easier to use than Windows.
Found category: software
Raw scores: {'carsharing': 31.328611, 'medical': 23.695553, 'education': 32.89378, 'agriculture': 25.077934, 'software': 52.776863}

Idea: A ride-sharing service for people with dogs.
Found category: carsharing
Raw scores: {'carsharing': 43.234314, 'medical': 27.288063, 'education': 35.321945, 'agriculture': 30.258034, 'software': 38.36947}

Idea: A service to match tutors to students.
Found category: education
Raw scores: {'carsharing': 33.708992, 'medical': 34.467037, 'education': 49.565353, 'agriculture': 27.424328, 'software': 40.24421}

Idea: A medical device for better pulse measuring.
Found category: medical
Raw scores: {'carsharing': 25.984545, 'medical': 53.09027, 'education': 34.158226, 'agriculture': 26.30721, 'software': 42.67244}

Idea: A low cost pesticide for paddy plantations.
Found category: agriculture
Raw scores: {'carsharing': 41.751984, 'medical': 27.86141, 'education': 30.812204,